In [ ]:
import random
import numpy as np
import tensorflow as tf
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

IMG_SIZE = (224, 224)
BATCH_SIZE = 16

In [2]:
import tensorflow as tf

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_i = tf.keras.utils.image_dataset_from_directory(r"C:\Users\inesb\Downloads\Deep_Learning\data\train",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode=None,
    shuffle=True)

val_i = tf.keras.utils.image_dataset_from_directory(r"C:\Users\inesb\Downloads\Deep_Learning\data\validation",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode=None,
    shuffle=False)

Found 9585 files.
Found 1199 files.


In [3]:
noise_factor = 0.2

def normalize_and_add_noise(x):
    x = tf.cast(x, tf.float32) / 255.0
    noise = tf.random.normal(shape=tf.shape(x)) * noise_factor
    x_noisy = tf.clip_by_value(x + noise, 0.0, 1.0)
    return x_noisy, x

In [4]:
noise_factor = 0.2

def normalize_and_add_noise(x):
    x = tf.cast(x, tf.float32) / 255.0
    noise = tf.random.normal(tf.shape(x)) * noise_factor
    x_noisy = tf.clip_by_value(x + noise, 0.0, 1.0)
    return x_noisy, x

In [5]:
noisy_train_i = (train_i.map(normalize_and_add_noise))

noisy_val_i = (val_i.map(normalize_and_add_noise))

In [6]:
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses

class Denoise(Model):
  def __init__(self):
    super(Denoise, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Input(shape=(28, 28, 3)),
      layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=2),
      layers.Conv2D(8, (3, 3), activation='relu', padding='same', strides=2)])

    self.decoder = tf.keras.Sequential([
      layers.Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same')])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Denoise()

In [ ]:
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

autoencoder.fit(noisy_train_i, validation_data = noisy_val_i, epochs = 15)

Epoch 1/15


215/300 ━━━━━━━━━━━━━━━━━━━━ 18s 219ms/step - loss: 0.0574

In [13]:
autoencoder.summary()
autoencoder.trainable = False

Model: "denoise"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 7, 7, 8)        │         1,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 224, 224, 1)    │         1,897 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,517 (41.09 KB)

 Trainable params: 3,505 (13.69 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 7,012 (27.39 KB)